# 필요한 라이브러리 설치

In [1]:
import numpy as np
import pandas as pd

In [4]:
!pip install transformers

In [18]:
from transformers import TFBertModel, BertTokenizer, TFBertForSequenceClassification

In [ ]:
pip install tensorflow==2.11

In [8]:
import tensorflow as tf
from tensorflow.keras import layers, Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.datasets import imdb
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [9]:
from tensorflow.keras import layers, Sequential

In [10]:
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.datasets import imdb
from tensorflow.keras.preprocessing.sequence import pad_sequences

# 데이터 전처리

In [27]:
df = pd.read_csv("https://raw.githubusercontent.com/songys/Chatbot_data/master/ChatbotData.csv")
df.head()

,Q,A,label
0,12시 땡!,하루가 또 가네요.,0
1,1지망 학교 떨어졌어,위로해 드립니다.,0
2,3박4일 놀러가고 싶다,여행은 언제나 좋죠.,0
3,3박4일 정도 놀러가고 싶다,여행은 언제나 좋죠.,0
4,PPL 심하네,눈살이 찌푸려지죠.,0


In [28]:
df_need = df[df.label != 0] # label이 0인 값 제거

In [29]:
df_shuffled = df_need.sample(frac=1).reset_index(drop=True)

In [30]:
df_shuffled.head()

,Q,A,label
0,잊혀진다는거,마음 아픈 일이기도 해요.,1
1,바람 나서 떠난 여친이 전화왓네.,사람 고쳐쓰는 거 아니에요.,1
2,몇번이나 매달려보셨어??,글쎄요. 기억나지 않네요.,1
3,본능적인 끌림,운명적이네요.,2
4,정말 모든게 그립지만.,묻어두는 것도 좋겠지요.,1


In [31]:
df_shuffled = df_shuffled.loc[:, ['Q', 'label']]

In [45]:
train_data = list(df_shuffled.Q)[:4000]
test_data = list(df_shuffled.Q)[4000:5000]
train_label = list(df_shuffled.label)[:4000]
test_label = list(df_shuffled.label)[4000:5000]

In [33]:
# 토크나이저 불러오기
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased', do_lower_case=False)

In [42]:
def encode(data, tokenizer):
    input_ids = []
    attention_masks = []
    token_type_ids = []
 
    for text in data:
        tokenized_text = tokenizer.encode_plus(text,
                                            max_length=50,
                                            add_special_tokens = True,
                                            pad_to_max_length=True,
                                            return_attention_mask=True,
                                              truncation=True)
        
        input_ids.append(tokenized_text['input_ids'])
        attention_masks.append(tokenized_text['attention_mask'])
        token_type_ids.append(tokenized_text['token_type_ids'])
    
    return input_ids, attention_masks, token_type_ids


In [46]:
#학습데이터 토큰화
train_input_ids, train_attention_masks, train_token_type_ids = encode(train_data, tokenizer)
 
#테스트데이터 토큰화
test_input_ids, test_attention_masks, test_token_type_ids = encode(test_data, tokenizer)

In [48]:
#딕셔너리 형태로 변환해서 출력 
def map_example_to_dict(input_ids, attention_masks, token_type_ids, label):
    return {
      "input_ids": input_ids,
      "token_type_ids": token_type_ids,
      "attention_mask": attention_masks,
      }, label
      
 #데이터를 BERT에 넣을 수 있는 형태로 변경 
def data_encode(input_ids_list, attention_mask_list, token_type_ids_list, label_list):
    return tf.data.Dataset.from_tensor_slices((input_ids_list, attention_mask_list, token_type_ids_list, label_list)).map(map_example_to_dict)

In [55]:
train_label = np.array(train_label) - 1

In [ ]:
test_label = np.array(train_label) - 1

In [56]:
train_label[:4]

array([0, 0, 0, 1])

In [ ]:
BATCH_SIZE = 32
 
#학습 데이터
train_data_encoded = data_encode(train_input_ids, train_attention_masks, train_token_type_ids,train_label).shuffle(10000).batch(BATCH_SIZE)
 
#평가 데이터
test_data_encoded = data_encode(test_input_ids, test_attention_masks, test_token_type_ids, test_label).batch(BATCH_SIZE)

# 필요한 라이브러리 설치

In [ ]:
!pip install -q opencv-python
!pip install -q -U "tensorflow-text==2.11.*"
!pip install -q tf-models-official


[notice] A new release of pip available: 22.3.1 -> 23.3.2
[notice] To update, run: pip install --upgrade pip


In [21]:
import numpy as np

# 모델 불러온 후 학습 코드

In [19]:
# 사용하지 않는 함수
def add_model_layer(model, num_words, embedding_len):
    # model은 from_pretrained으로 불러온 모델...
    model.add(layers.SimpleRNN(16))
    model.add(layers.Dense(1, activation='sigmoid'))
    
    return model

# 층을 더 쌓아서 추가학습을 시키려면 이렇게 해야할까 생각했는데 비슷한 방법을 찾지 못했습니다
# 잘못 이해하고 있는 점에 대해 알고 싶습니다!

In [65]:
model = TFBertForSequenceClassification.from_pretrained(
    "bert-base-uncased", # Use the 12-layer BERT model, with an uncased vocab.
    num_labels = 2, # The number of output labels--2 for binary classification.
#                     # You can increase this for multi-class tasks.   
#     output_attentions = False, # Whether the model returns attentions weights.
#     output_hidden_states = False, # Whether the model returns all hidden-states.
)

All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [73]:
model.summary()

Model: "tf_bert_for_sequence_classification_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bert (TFBertMainLayer)      multiple                  109482240 
                                                                 
 dropout_151 (Dropout)       multiple                  0         
                                                                 
 classifier (Dense)          multiple                  1538      
                                                                 
Total params: 109,483,778
Trainable params: 109,483,778
Non-trainable params: 0
_________________________________________________________________


In [72]:
train_data_encoded

<BatchDataset element_spec=({'input_ids': TensorSpec(shape=(None, 50), dtype=tf.int32, name=None), 'token_type_ids': TensorSpec(shape=(None, 50), dtype=tf.int32, name=None), 'attention_mask': TensorSpec(shape=(None, 50), dtype=tf.int32, name=None)}, TensorSpec(shape=(None,), dtype=tf.int64, name=None))>

In [75]:
optimizer = tf.keras.optimizers.Adam(1e-5)
loss = tf.keras.losses.BinaryCrossentropy(from_logits=True)
metric = tf.keras.metrics.SparseCategoricalAccuracy('accuracy')
model.compile(optimizer=optimizer, loss=loss, metrics=[metric])
NUM_EPOCHS = 10
history = model.fit(train_data_encoded, epochs=NUM_EPOCHS, batch_size=BATCH_SIZE, validation_data=test_data_encoded)

Epoch 1/10


ValueError: in user code:

    File "/home/elicer/.local/lib/python3.10/site-packages/keras/engine/training.py", line 1249, in train_function  *
        return step_function(self, iterator)
    File "/home/elicer/.local/lib/python3.10/site-packages/keras/engine/training.py", line 1233, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/home/elicer/.local/lib/python3.10/site-packages/keras/engine/training.py", line 1222, in run_step  **
        outputs = model.train_step(data)
    File "/home/elicer/.local/lib/python3.10/site-packages/transformers/modeling_tf_utils.py", line 1675, in train_step
        loss = self.compiled_loss(y, y_pred, sample_weight, regularization_losses=self.losses)
    File "/home/elicer/.local/lib/python3.10/site-packages/keras/engine/compile_utils.py", line 265, in __call__
        loss_value = loss_obj(y_t, y_p, sample_weight=sw)
    File "/home/elicer/.local/lib/python3.10/site-packages/keras/losses.py", lin

Epoch 1/10
---------------------------------------------------------------------------
ValueError                                Traceback (most recent call last)
Cell In[75], line 6
      4 model.compile(optimizer=optimizer, loss=loss, metrics=[metric])
      5 NUM_EPOCHS = 10
----> 6 history = model.fit(train_data_encoded, epochs=NUM_EPOCHS, batch_size=BATCH_SIZE, validation_data=test_data_encoded)

File ~/.local/lib/python3.10/site-packages/keras/utils/traceback_utils.py:70, in filter_traceback.<locals>.error_handler(*args, **kwargs)
     67     filtered_tb = _process_traceback_frames(e.__traceback__)
     68     # To get the full stack trace, call:
     69     # `tf.debugging.disable_traceback_filtering()`
---> 70     raise e.with_traceback(filtered_tb) from None
     71 finally:
     72     del filtered_tb

File /tmp/__autograph_generated_file9j0jpqer.py:15, in outer_factory.<locals>.inner_factory.<locals>.tf__train_function(iterator)
     13 try:
     14     do_return = True
---> 15     retval_ = ag__.converted_call(ag__.ld(step_function), (ag__.ld(self), ag__.ld(iterator)), None, fscope)
     16 except:
     17     do_return = False

File ~/.local/lib/python3.10/site-packages/transformers/modeling_tf_utils.py:1675, in TFPreTrainedModel.train_step(self, data)
   1672             y_pred = y_pred[0]
   1674     if loss is None:
...
    File "/home/elicer/.local/lib/python3.10/site-packages/keras/backend.py", line 5680, in binary_crossentropy
        return tf.nn.sigmoid_cross_entropy_with_logits(

    ValueError: `logits` and `labels` must have the same shape, received ((None, 2) vs (None, 1)).
Output is truncated. View as a scrollable element or open in a text editor. Adjust cell output settings...